In [0]:
source_path = "path_to_bronze"
destination_path = "path_to_silver"

In [0]:
dbutils.fs.ls(f"{source_path}") 

In [0]:
df = spark.read.format("parquet").load(f"{source_path}/Address/Address.parquet")
df.show()

+---------+--------------------+------------+-----------+----------------+-------------+----------+--------------------+-------------------+
|AddressID|        AddressLine1|AddressLine2|       City|   StateProvince|CountryRegion|PostalCode|             rowguid|       ModifiedDate|
+---------+--------------------+------------+-----------+----------------+-------------+----------+--------------------+-------------------+
|        9|   8713 Yosemite Ct.|        NULL|    Bothell|      Washington|United States|     98011|268af621-76d7-4c7...|2006-07-01 00:00:00|
|       11| 1318 Lasalle Street|        NULL|    Bothell|      Washington|United States|     98011|981b3303-aca2-49c...|2007-04-01 00:00:00|
|       25|    9178 Jumping St.|        NULL|     Dallas|           Texas|United States|     75201|c8df3bd9-48f0-465...|2006-09-01 00:00:00|
|       28|    9228 Via Del Sol|        NULL|    Phoenix|         Arizona|United States|     85004|12ae5ee1-fc3e-468...|2005-09-01 00:00:00|
|       32|  

In [0]:
from pyspark.sql.functions import from_utc_timestamp, date_format
from pyspark.sql.types import TimestampType

df = df.withColumn("ModifiedDate", date_format(from_utc_timestamp(df['ModifiedDate'].cast(TimestampType()), "UTC"), "yyyy-MM-dd"))

In [0]:
df.show()

+---------+--------------------+------------+-----------+----------------+-------------+----------+--------------------+------------+
|AddressID|        AddressLine1|AddressLine2|       City|   StateProvince|CountryRegion|PostalCode|             rowguid|ModifiedDate|
+---------+--------------------+------------+-----------+----------------+-------------+----------+--------------------+------------+
|        9|   8713 Yosemite Ct.|        NULL|    Bothell|      Washington|United States|     98011|268af621-76d7-4c7...|  2006-07-01|
|       11| 1318 Lasalle Street|        NULL|    Bothell|      Washington|United States|     98011|981b3303-aca2-49c...|  2007-04-01|
|       25|    9178 Jumping St.|        NULL|     Dallas|           Texas|United States|     75201|c8df3bd9-48f0-465...|  2006-09-01|
|       28|    9228 Via Del Sol|        NULL|    Phoenix|         Arizona|United States|     85004|12ae5ee1-fc3e-468...|  2005-09-01|
|       32|   26910 Indela Road|        NULL|   Montreal|     

Date Transformations for tables

In [0]:
table_name = []
for i in dbutils.fs.ls(f"{source_path}"):
    table_name.append(i.name.split("/")[0])

In [0]:
for i in table_name:
    path = f'{source_path}'+ i + '/' + i + '.parquet'
    df = spark.read.format("parquet").load(path)
    column = df.columns

    for col in column:
        if "Date" in col or "date" in col:
            df = df.withColumn(col, date_format(from_utc_timestamp(df[col].cast(TimestampType()), "UTC"), "yyyy-MM-dd"))
    output_path = f'{destination_path}'+ i + '/'
    df.write.format('delta').mode('overwrite').save(output_path)
